# Query Translation: Decomposition

![decomposition](../images/images-decomposition.png)

**Decomposition** is a technique to handle complex or multi-part queries by breaking them down into smaller, simpler pieces. Each part is processed independently, and the results are combined to generate a final response. Let’s make this simple:

### The Problem Decomposition Solves
When you ask a RAG system a question that has multiple parts or is too complex, the system might:
- Struggle to understand the full intent.
- Retrieve information that is incomplete or irrelevant because it can’t focus on each aspect of the query.

**Decomposition** addresses this by splitting the query into manageable pieces, so each part can be processed more effectively.

### How Decomposition Works
1. **Break Down the Query**:
   - The system splits the original query into smaller, independent sub-queries. Each sub-query targets one specific aspect of the original question.
   - Example:
     - Original Query: "What are the best practices for securing APIs, and how does OAuth 2.0 work?"
     - Sub-Queries:
       - "What are the best practices for securing APIs?"
       - "How does OAuth 2.0 work?"

2. **Retrieve for Each Sub-Query**:
   - Each sub-query is sent to the retriever to fetch relevant documents or information.
   - Example:
     - For "best practices for securing APIs," it retrieves documents on general API security.
     - For "how OAuth 2.0 works," it retrieves documents explaining the OAuth 2.0 framework.

3. **Combine Retrieved Information**:
   - The retrieved documents or information for all sub-queries are combined into a single context for the next step.

4. **Generate the Response**:
   - The combined information is passed to the LLM, which uses it to generate a comprehensive and coherent response to the original query.

### Why Decomposition Works
- **Simplifies Complexity**: By breaking a complex question into simpler parts, it becomes easier to retrieve relevant information for each part.
- **Improves Focus**: Each sub-query is more focused, reducing the chances of retrieving irrelevant documents.
- **Combines Expertise**: Allows the system to handle different aspects of a query with equal attention, leading to more complete answers.

### Simple Example
#### Query:
*"What are the top API security measures, and how do they differ for REST and GraphQL APIs?"*

#### Without Decomposition:
- The system might retrieve general API security documents without addressing the specific REST vs. GraphQL distinction.

#### With Decomposition:
1. Sub-Queries:
   - "What are the top API security measures?"
   - "What are the security considerations for REST APIs?"
   - "What are the security considerations for GraphQL APIs?"

2. Retrieved Documents:
   - For Sub-Query 1: General security guidelines like OAuth, input validation.
   - For Sub-Query 2: REST-specific guidelines like rate limiting.
   - For Sub-Query 3: GraphQL-specific considerations like query depth limiting.

3. Combined Information:
   - A mix of general security practices and specific guidance for REST and GraphQL.

4. Final Output:
   - A response like: "Top API security measures include OAuth 2.0, input validation, and HTTPS enforcement. For REST APIs, rate limiting and endpoint validation are crucial, while for GraphQL APIs, query depth and complexity limiting are important to prevent abuse."

### Key Benefits of Decomposition
- **Handles Complexity**: Effectively answers multi-part or detailed queries.
- **Complete Responses**: Ensures all aspects of the query are addressed.
- **Better Retrieval**: Each sub-query retrieves more focused and relevant results.

In short, **Decomposition** is like breaking a big question into smaller, easier-to-answer questions, retrieving information for each, and piecing it all together to provide a thorough response!

![decomposition_recursive](../images/decomposition_recursive.png)

Arxiv papers:

- [Least-to-Most Prompting Enables Complex Reasoning in Large Language Models](https://arxiv.org/pdf/2205.10625)
- [Interleaving Retrieval with Chain-of-Thought Reasoning for Knowledge-Intensive Multi-Step Questions](https://arxiv.org/pdf/2212.10509)

## Setup

In [17]:
%run "../Z - Common/setup.ipynb"

In [18]:
docs = load_sample_data()
split_docs = split_sample_data(docs)
retriever = seed_sample_data(split_docs)

Let's start by creating the prompt and chain to generate the initial decomposed sub-questions:


In [19]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

template = """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation. \n
Generate 5 search queries related to: {question} \n
Return the queries only as text only, each separated with a new line character."""

prompt_generate_queries = ChatPromptTemplate.from_template(template)

chain_generate_queries = ( 
                          prompt_generate_queries 
                          | llm
                          | StrOutputParser() 
                          | (lambda x: x.split("\n")))

In [20]:
question = "What are the main components of an LLM-powered autonomous agent system?"
decomposed_questions = chain_generate_queries.invoke({"question":question})

decomposed_questions

['components of LLM agent architecture',
 'difference between LLM and autonomous agent system',
 'key modules needed for LLM agent decision making',
 'memory and planning systems in LLM agents',
 'tools and APIs integration in autonomous LLM agents']

Define the prompt to answer the original question based on the additional context gathered via the decomposition chain:

In [21]:
template = """Here is the question you need to answer:

\n --- \n {question} \n --- \n

Here is any available background question + answer pairs:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question: 

\n --- \n {context} \n --- \n

Use the above context and any background question + answer pairs to answer the question: \n {question}
"""

prompt_solve_recursively = ChatPromptTemplate.from_template(template)

Define an additional chain that takes the `decomposed_questions` and solves using the above prompt.

In [22]:
def format_qa_pair_recursive(question, answer):
    """Format Q and A pair"""
    
    formatted_string = ""
    formatted_string += f"Question: {question}\nAnswer: {answer}\n\n"
    return formatted_string.strip()

In [23]:
from operator import itemgetter

def retrieve_and_rag_recursive(decomposed_questions):
    q_a_pairs = ""
    for q in decomposed_questions:
        
        rag_chain = (
            {"context": itemgetter("question") | retriever, 
            "question": itemgetter("question"),
            "q_a_pairs": itemgetter("q_a_pairs")} 
            | prompt_solve_recursively
            | llm
            | StrOutputParser())

        answer = rag_chain.invoke({"question":q,"q_a_pairs":q_a_pairs})
        q_a_pair = format_qa_pair_recursive(q,answer)
        q_a_pairs = q_a_pairs + "\n---\n"+  q_a_pair
        
    return (answer, q_a_pairs)

In [24]:
result_recursive,q_a_pairs = retrieve_and_rag_recursive(decomposed_questions)

print(result_recursive)
print(q_a_pairs)

Based on the available context and background information, I'll explain tools and APIs integration in autonomous LLM agents:

Key Aspects of Tool and API Integration:

1. Core Integration Components
- API connections to external tools and services
- Tool selection and usage logic
- Output parsing and handling mechanisms
- Interface management between LLM and external tools

2. Functional Integration
- Allows the LLM agent to interact with external systems
- Enables real-world actions beyond just language processing
- Provides access to additional capabilities and data sources
- Helps bridge the gap between reasoning and actual task execution

3. Integration Architecture
- Tool selection module to choose appropriate tools for tasks
- API connectors to establish connections with external services
- Output handlers to process and interpret tool responses
- Error handling mechanisms for failed interactions
- Data formatters to ensure proper communication between systems

4. Key Benefits
- 

Let's now take a look at how to perform decomposition individually.

![decomposition_individually](../images/decomposition_individually.png)

In [25]:
def retrieve_and_rag_individually(question,prompt_rag,sub_question_generator_chain):
    """RAG on each sub-question"""
    
    # Use our decomposition / 
    sub_questions = sub_question_generator_chain.invoke({"question":question})
    
    # Initialize a list to hold RAG chain results
    rag_results = []
    
    for sub_question in sub_questions:
        
        # Retrieve documents for each sub-question
        retrieved_docs = retriever.invoke(sub_question)
        
        # Use retrieved documents and sub-question in RAG chain
        answer = (
            prompt_rag 
            | llm
            | StrOutputParser()).invoke({"context": retrieved_docs, "question": sub_question})
        rag_results.append(answer)
    
    return rag_results,sub_questions

In [27]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

answers, questions = retrieve_and_rag_individually(question, prompt, chain_generate_queries)

print("Answers:")
print(answers)

print("Questions:")
print(questions)

/Users/deanhart/miniconda3/envs/rag-techniques/lib/python3.11/site-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Answers:
['I apologize, but I cannot provide a complete answer about the components of LLM agent architecture based on the given context. The provided context only mentions that there are limitations to LLM-centered agents but does not describe their components.', 'LLM agents use large language models as their core controller or "brain," while traditional AI agents typically rely on more rigid, rule-based systems. LLM agents can break down complex tasks into subgoals and have the ability to perform self-reflection and learning from mistakes. Additionally, LLM agents are more flexible general problem solvers that can handle a wider range of tasks, from writing to programming to complex reasoning.', "Based on the provided context, I cannot provide specific details about memory systems in LLM-powered autonomous agents. While the context mentions memory as one of the key components of LLM-powered autonomous agent systems, it doesn't elaborate on the memory systems themselves.", "LLM agents

Build the final chain to perform decomposition individually.

In [28]:
#### QUERY TRANSLATION - Decomposition - utility function ####

def format_qa_pairs_individually(questions, answers):
    """Format Q and A pairs"""
    
    formatted_string = ""
    for i, (question, answer) in enumerate(zip(questions, answers), start=1):
        formatted_string += f"Question {i}: {question}\nAnswer {i}: {answer}\n\n"
    return formatted_string.strip()

context = format_qa_pairs_individually(questions, answers)

In [29]:
#### QUERY TRANSLATION - Decomposition - prompt an chain individually ####

# Prompt
template = """Here is a set of Q+A pairs:

{context}

Use these to synthesize an answer to the question: {question}
"""

prompt_solve_individually = ChatPromptTemplate.from_template(template)

chain_solve_individually = (
    prompt_solve_individually
    | llm
    | StrOutputParser()
)

In [30]:
from pprint import pprint

result_individually=chain_solve_individually.invoke({"context":context,"question":question})

pprint(result_individually)

('Based on the Q+A pairs provided, I can synthesize that LLM-powered '
 'autonomous agent systems have several main components:\n'
 '\n'
 '1. Core Controller/Brain: The LLM itself serves as the central controller, '
 'providing more flexible and general problem-solving capabilities compared to '
 'traditional rule-based systems.\n'
 '\n'
 '2. Planning and Reasoning Modules: These can be implemented in different '
 'ways:\n'
 '- ReAct approach: Integrates reasoning and acting through combined '
 'task-specific actions and language-based reasoning\n'
 '- LLM+P approach: Uses external classical planners with PDDL for '
 'long-horizon planning\n'
 '\n'
 "3. Memory Systems: While specific details aren't provided, memory is "
 'mentioned as a key component of these systems.\n'
 '\n'
 '4. Task Management: The system can break down complex tasks into subgoals '
 'and includes capabilities for self-reflection and learning from mistakes.\n'
 '\n'
 "However, it's worth noting that some aspects, s

Save the results so we can compare later.

In [31]:
write_results("decomposition-recursive.txt", result_recursive)
write_results("decomposition-individually.txt", result_individually)